### ORBIT Example - ParametricManager

ParametricManager provides a similar interface into ORBIT as ProjectManager but allows for some (or all) inputs to be parameterized. This class is useful for quickly exploring how a module or project scales with certain inputs.

In [12]:
from ORBIT.phases.design import MonopileDesign, ElectricalDesign

from ORBIT import ParametricManager, ProjectManager

import numpy as np

### 1. Monopile Design Example
Perform a parametric sweep of site depth and mean wind speed to see the effects on monopile capex.

In [13]:
# For this example we will look at the MonopileDesign module.
MonopileDesign.expected_config

{'site': {'depth': 'm', 'mean_windspeed': 'm/s'},
 'plant': {'num_turbines': 'int'},
 'turbine': {'rotor_diameter': 'm',
  'hub_height': 'm',
  'rated_windspeed': 'm/s'},
 'monopile_design': {'yield_stress': 'Pa (optional)',
  'load_factor': 'float (optional)',
  'material_factor': 'float (optional)',
  'monopile_density': 'kg/m3 (optional)',
  'monopile_modulus': 'Pa (optional)',
  'monopile_tp_connection_thickness': 'm (optional)',
  'transition_piece_density': 'kg/m3 (optional)',
  'transition_piece_thickness': 'm (optional)',
  'transition_piece_length': 'm (optional)',
  'soil_coefficient': 'N/m3 (optional)',
  'air_density': 'kg/m3 (optional)',
  'weibull_scale_factor': 'float (optional)',
  'weibull_shape_factor': 'float (optional)',
  'turb_length_scale': 'm (optional)',
  'monopile_steel_cost': 'USD/t (optional)',
  'tp_steel_cost': 'USD/t (optional)'}}

In [14]:
# The following inputs are the only "required" inputs for the MonopileDesign module.
# Thee 'site' inputs are commented out as they will be defined as parametric inputs below.

base_config = {
#     "site": {
#         "depth": 20,
#         "mean_windspeed": 8
#     }
    "turbine": "12MW_generic",
    "plant": {
        "num_turbines": 50
    }
}

In [15]:
# Parametric inputs:

parameters = {
    "site.depth": [20, 40, 60],           # The dot-notation allows you to access nested dictionaries
    "site.mean_windspeed": [8, 9, 10]     # These inputs correspond to the 'depth' and 'mean_windspeed' inputs above, which are nested
}                                         # in the 'site' dictionary.

In [16]:
# Desired results:
# Since there are so many available results in ORBIT, you have to tell ParametricManager which ones
# you are interested in for this parametric run. The syntax for this always follows the
# 'lambda run: run.{output}' format. The {output} can be any output available for the configured module

results = {
    "capex": lambda run: run.total_cost
}

In [17]:
# Pass the above definitions into ParametricManager like this:
# Note: I only want to run MonopileDesign so I passed it into module directily.
# If you want to run an entire project (with multiple modules), leave module blank and it will
# automatically run ProjectManager

parametric = ParametricManager(base_config, parameters, results, module=MonopileDesign)
parametric.run()


# By default, ParametricManager doesn't run the product of all parameters and just
# runs the first set of inputs, then the second set of inputs, etc.
# In this case, there will only be 3 results
parametric.results

,site.depth,site.mean_windspeed,capex
0,20,8,2.388521e+08
1,40,9,3.261239e+08
2,60,10,4.289900e+08


In [18]:
# To configure ParametricManager to run the product of all inputs, pass 'product=True'
# This will result in 3x3 results as each combination is ran
parametric = ParametricManager(base_config, parameters, results, module=MonopileDesign, product=True)
parametric.run()

parametric.results

,site.depth,site.mean_windspeed,capex
0,20,8,2.388521e+08
1,20,9,2.545394e+08
2,20,10,2.705817e+08
3,40,8,3.062888e+08
4,40,9,3.261239e+08
5,40,10,3.463884e+08
6,60,8,3.798072e+08
7,60,9,4.041444e+08
8,60,10,4.289900e+08


In [19]:
# If you are configuring many parameters it can take a long time to run, especially if weather is turned on.
# To get an idea of how long it'll take, you can run the .preview() method:

parameters = {
    "site.depth": [20, 40, 60],
    "site.mean_windspeed": [8, 9, 10],
    "monopile_design.soil_coefficient": [4000000, 4500000, 5000000]
}

parametric = ParametricManager(base_config, parameters, results, module=MonopileDesign, product=True)
parametric.preview()

10 runs elapsed time: 0.02s
27 runs estimated time: 0.06s


,site.depth,site.mean_windspeed,monopile_design.soil_coefficient,capex
0,40,9,4000000,3.261239e+08
1,60,9,4000000,4.041444e+08
2,60,8,4000000,3.798072e+08
3,20,8,4500000,2.371472e+08
4,20,9,4500000,2.526935e+08
5,60,8,5000000,3.758376e+08
6,40,10,5000000,3.421996e+08
7,40,8,5000000,3.027043e+08
8,40,8,4000000,3.062888e+08
9,60,9,5000000,3.998461e+08


### 2. Electrical Export Design

Perform a parametric sweep of export cable technology (HVDC or HVAC) and how plant capacity impacts the cable capex.

In [20]:
ElectricalDesign.expected_config

{'site': {'distance_to_landfall': 'km', 'depth': 'm'},
 'landfall': {'interconnection_distance': 'km (optional)'},
 'plant': {'capacity': 'MW'},
 'export_system_design': {'cables': 'str',
  'num_redundant': 'int (optional)',
  'touchdown_distance': 'm (optional, default: 0)',
  'percent_added_length': 'float (optional)',
  'cable_crossings': {'crossing_number': 'int (optional)',
   'crossing_unit_cost': 'float (optional)'}},
 'substation_design': {'mpt_cost': 'USD (optional)',
  'topside_fab_cost_rate': 'USD/t (optional)',
  'topside_design_cost': 'USD (optional)',
  'shunt_cost_rate': 'USD/MW (optional)',
  'switchgear_cost': 'USD (optional)',
  'dc_breaker_cost': 'USD (optional)',
  'backup_gen_cost': 'USD (optional)',
  'workspace_cost': 'USD (optional)',
  'other_ancillary_cost': 'USD (optional)',
  'converter_cost': 'USD (optional)',
  'onshore_converter_cost': 'USD (optional)',
  'topside_assembly_factor': 'float (optional)',
  'oss_substructure_cost_rate': 'USD/t (optional)',
  

In [21]:
base_config = {
    'export_cable_install_vessel': 'example_cable_lay_vessel',
    'site': {
        'distance': 100,
        'depth': 20,
        'distance_to_landfall': 50
    },
    'plant': {
        'turbine_rating': 10
#          'capacity': 500
    },
    'turbine': {'turbine_rating': 10},
    'oss_install_vessel': 'example_heavy_lift_vessel',
    'feeder': 'future_feeder',
#     'export_system_design': {
#         'cables': 'XLPE_500mm_220kV',
#     }
}

In [22]:
# Parameters
parameters = {
    'export_system_design.cables': ['XLPE_1000mm_220kV', 'HVDC_2000mm_320kV'],
    'plant.capacity': np.arange(100,2100,100)
}

In [23]:
# Desired results

results = {
    'cable_cost': lambda run: run.total_cable_cost,
    'oss_cost': lambda run: run.substation_cost,
    'num_cables': lambda run: run.num_cables,
    'num_substations': lambda run: run.num_substations,
}

In [24]:
# Follow the same steps as show in Example 1

parametric = ParametricManager(base_config, parameters, results, module = ElectricalDesign, product=True)
parametric.run()
parametric.results

,export_system_design.cables,plant.capacity,cable_cost,oss_cost,num_cables,num_substations
0,XLPE_1000mm_220kV,100,75288400.0,1.532619e+07,1,1
1,XLPE_1000mm_220kV,200,75288400.0,1.532619e+07,1,1
2,XLPE_1000mm_220kV,300,75288400.0,1.532619e+07,1,1
3,XLPE_1000mm_220kV,400,150576800.0,2.420237e+07,2,1
4,XLPE_1000mm_220kV,500,150576800.0,2.420237e+07,2,1
5,XLPE_1000mm_220kV,600,150576800.0,2.420237e+07,2,1
6,XLPE_1000mm_220kV,700,225865200.0,3.307856e+07,3,1
7,XLPE_1000mm_220kV,800,225865200.0,3.307856e+07,3,1
8,XLPE_1000mm_220kV,900,225865200.0,3.307856e+07,3,1
9,XLPE_1000mm_220kV,1000,301153600.0,4.195474e+07,4,1
